In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/churn-bigml-80.csv')
print(df.shape)
df.head()

(2666, 20)


,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   2666 non-null   object 
 1   Account length          2666 non-null   int64  
 2   Area code               2666 non-null   int64  
 3   International plan      2666 non-null   object 
 4   Voice mail plan         2666 non-null   object 
 5   Number vmail messages   2666 non-null   int64  
 6   Total day minutes       2666 non-null   float64
 7   Total day calls         2666 non-null   int64  
 8   Total day charge        2666 non-null   float64
 9   Total eve minutes       2666 non-null   float64
 10  Total eve calls         2666 non-null   int64  
 11  Total eve charge        2666 non-null   float64
 12  Total night minutes     2666 non-null   float64
 13  Total night calls       2666 non-null   int64  
 14  Total night charge      2666 non-null   

In [5]:
df.isna().sum()

State                     0
Account length            0
Area code                 0
International plan        0
Voice mail plan           0
Number vmail messages     0
Total day minutes         0
Total day calls           0
Total day charge          0
Total eve minutes         0
Total eve calls           0
Total eve charge          0
Total night minutes       0
Total night calls         0
Total night charge        0
Total intl minutes        0
Total intl calls          0
Total intl charge         0
Customer service calls    0
Churn                     0
dtype: int64

In [6]:
df['Churn'].value_counts(normalize=True)

Churn
False    0.854464
True     0.145536
Name: proportion, dtype: float64

In [7]:
LEAKY_COLS = [
    "Total day charge",
    "Total eve charge",
    "Total night charge",
    "Total intl charge",
]

In [8]:
df = df.drop(columns=LEAKY_COLS)

# 3) Standardize categorical text
def normalize_yes_no(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .str.lower()
         .replace({"true": "yes", "false": "no"})
    )


In [9]:
df["International plan"] = normalize_yes_no(df["International plan"])
df["Voice mail plan"] = normalize_yes_no(df["Voice mail plan"])

df["State"] = df["State"].astype(str).str.strip().str.upper()

# 4) Treat Area code as categorical (string)
df["Area code"] = df["Area code"].astype(str).str.strip()

In [10]:
# 5) Check results
print("Shape after drop:", df.shape)
print("\nUnique values - International plan:", df["International plan"].unique())
print("Unique values - Voice mail plan:", df["Voice mail plan"].unique())
print("Area code values:", sorted(df["Area code"].unique())[:10], "...")
print("State sample:", df["State"].unique()[:10])

Shape after drop: (2666, 16)

Unique values - International plan: ['no' 'yes']
Unique values - Voice mail plan: ['yes' 'no']
Area code values: ['408', '415', '510'] ...
State sample: ['KS' 'OH' 'NJ' 'OK' 'AL' 'MA' 'MO' 'WV' 'RI' 'IA']


In [11]:
df.shape

(2666, 16)

In [14]:
df["International plan"].unique()


array(['no', 'yes'], dtype=object)

In [15]:
df["Voice mail plan"].unique()

array(['yes', 'no'], dtype=object)

In [13]:
df["State"].nunique()


51

In [16]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [17]:
# -----------------------------
# Separate features and target
# -----------------------------
target = "Churn"
X = df.drop(columns=[target])
y = df[target].astype(int)  # Convert boolean target to 0/1

In [18]:
categorical_features = [
    "State",
    "Area code",
    "International plan",
    "Voice mail plan"
]

numeric_features = [col for col in X.columns if col not in categorical_features]


In [19]:
# -----------------------------
# Preprocessing pipeline
# - Scale numeric features
# - One-hot encode categorical features
# -----------------------------
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

In [20]:
# -----------------------------
# Baseline model
# - Logistic Regression is interpretable and a strong baseline
# - class_weight handles class imbalance
# -----------------------------
model = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    random_state=42
)


In [21]:

# -----------------------------
# Full pipeline (preprocessing + model)
# -----------------------------
clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model)
])


In [22]:
# -----------------------------
# Cross-validation strategy
# - Stratified to preserve churn ratio
# -----------------------------
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)


In [23]:
# -----------------------------
# Evaluate using ROC-AUC
# -----------------------------
roc_auc_scores = cross_val_score(
    clf,
    X,
    y,
    cv=cv,
    scoring="roc_auc"
)

In [24]:
print("ROC-AUC CV mean:", roc_auc_scores.mean())
print("ROC-AUC CV std:", roc_auc_scores.std())

ROC-AUC CV mean: 0.8048282464274367
ROC-AUC CV std: 0.029967703287182113


In [28]:
import numpy as np
from sklearn.model_selection import cross_val_predict

# -----------------------------
# Get out-of-fold predicted probabilities
# -----------------------------
y_proba = cross_val_predict(
    clf,
    X,
    y,
    cv=cv,
    method="predict_proba"
)[:, 1]

In [29]:
def expected_cost(y_true, y_pred, cost_fn=100, cost_fp=10):
    """
    Compute expected business cost.
    
    cost_fn: cost of missing a churned customer
    cost_fp: cost of contacting a non-churned customer
    """
    fn = ((y_true == 1) & (y_pred == 0)).sum()
    fp = ((y_true == 0) & (y_pred == 1)).sum()
    tp = ((y_true == 1) & (y_pred == 1)).sum()
    
    total_cost = fn * cost_fn + (fp + tp) * cost_fp
    return total_cost

In [30]:
thresholds = np.linspace(0.01, 0.99, 99)
costs = []

for t in thresholds:
    y_pred = (y_proba >= t).astype(int)
    cost = expected_cost(y, y_pred)
    costs.append(cost)

best_threshold = thresholds[np.argmin(costs)]
best_cost = min(costs)

print("Best threshold:", round(best_threshold, 3))
print("Minimum expected cost:", best_cost)


Best threshold: 0.34
Minimum expected cost: 18120


In [31]:
default_pred = (y_proba >= 0.5).astype(int)
default_cost = expected_cost(y, default_pred)

print("Cost with default threshold (0.5):", default_cost)
print("Cost reduction (%):",
      round((default_cost - best_cost) / default_cost * 100, 2))


Cost with default threshold (0.5): 19040
Cost reduction (%): 4.83


In [33]:
# -----------------------------
# Load test (future / hold-out data)
# -----------------------------
df_test = pd.read_csv("../data/churn-bigml-20.csv")

# Drop leakage-prone columns (same as train)
df_test = df_test.drop(columns=LEAKY_COLS)

# Apply the same categorical normalization
df_test["International plan"] = normalize_yes_no(df_test["International plan"])
df_test["Voice mail plan"] = normalize_yes_no(df_test["Voice mail plan"])
df_test["State"] = df_test["State"].astype(str).str.strip().str.upper()
df_test["Area code"] = df_test["Area code"].astype(str).str.strip()

# Separate X_test and y_test
X_test = df_test.drop(columns=["Churn"])
y_test = df_test["Churn"].astype(int)

print("Test shape:", X_test.shape)
print("Churn rate (test):", y_test.mean())


Test shape: (667, 15)
Churn rate (test): 0.1424287856071964


In [34]:
# -----------------------------
# Fit final model on full training data (80%)
# -----------------------------
clf.fit(X, y)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Account length',
                                                   'Number vmail messages',
                                                   'Total day minutes',
                                                   'Total day calls',
                                                   'Total eve minutes',
                                                   'Total eve calls',
                                                   'Total night minutes',
                                                   'Total night calls',
                                                   'Total intl minutes',
                                                   'Total intl calls',
                                                   'Customer service calls']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['State', 'Area code',
                                                   'International plan',
                                                   'Voice mail plan'])])),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=2000,
                                    random_state=42))])

In [35]:
# -----------------------------
# Predict probabilities on test data
# -----------------------------
y_test_proba = clf.predict_proba(X_test)[:, 1]

# Apply business-optimized threshold
OPTIMAL_THRESHOLD = 0.34
y_test_pred = (y_test_proba >= OPTIMAL_THRESHOLD).astype(int)


In [36]:
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix
)

print("Test ROC-AUC:", roc_auc_score(y_test, y_test_proba))
print("\nClassification report (threshold = 0.34):")
print(classification_report(y_test, y_test_pred))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_test_pred))


Test ROC-AUC: 0.8187154950312845

Classification report (threshold = 0.34):
              precision    recall  f1-score   support

           0       0.97      0.61      0.75       572
           1       0.27      0.87      0.41        95

    accuracy                           0.64       667
   macro avg       0.62      0.74      0.58       667
weighted avg       0.87      0.64      0.70       667

Confusion matrix:
[[347 225]
 [ 12  83]]


In [37]:
test_cost = expected_cost(y_test, y_test_pred)

# For comparison: default threshold
default_test_pred = (y_test_proba >= 0.5).astype(int)
default_test_cost = expected_cost(y_test, default_test_pred)

print("Test cost (optimized threshold):", test_cost)
print("Test cost (default threshold 0.5):", default_test_cost)

print(
    "Cost reduction on test (%):",
    round((default_test_cost - test_cost) / default_test_cost * 100, 2)
)


Test cost (optimized threshold): 4280
Test cost (default threshold 0.5): 4200
Cost reduction on test (%): -1.9


In [41]:
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp

# -----------------------------
# Helper: Population Stability Index (PSI)
# -----------------------------
def psi(expected, actual, bins=10, eps=1e-6):
    """
    PSI between two distributions.
    expected: train series
    actual: test series
    """
    expected = pd.Series(expected).dropna()
    actual = pd.Series(actual).dropna()

    # Quantile-based bins from expected (train)
    quantiles = np.linspace(0, 1, bins + 1)
    breakpoints = np.unique(np.quantile(expected, quantiles))

    # If too few unique breakpoints, fallback to equal-width
    if len(breakpoints) <= 2:
        breakpoints = np.linspace(expected.min(), expected.max(), bins + 1)

    exp_counts, _ = np.histogram(expected, bins=breakpoints)
    act_counts, _ = np.histogram(actual, bins=breakpoints)

    exp_perc = exp_counts / max(exp_counts.sum(), 1)
    act_perc = act_counts / max(act_counts.sum(), 1)

    exp_perc = np.clip(exp_perc, eps, 1)
    act_perc = np.clip(act_perc, eps, 1)

    return np.sum((exp_perc - act_perc) * np.log(exp_perc / act_perc))

# -----------------------------
# Build drift report (numeric: PSI+KS, categorical: PSI on proportions)
# -----------------------------
train_df = df.copy()
test_df = df_test.copy()

target = "Churn"

X_train = train_df.drop(columns=[target])
X_test = test_df.drop(columns=[target])

categorical_features = ["State", "Area code", "International plan", "Voice mail plan"]
numeric_features = [c for c in X_train.columns if c not in categorical_features]

drift_rows = []

# Numeric drift: PSI + KS-test
for col in numeric_features:
    psi_val = psi(X_train[col], X_test[col], bins=10)
    ks_stat, ks_p = ks_2samp(X_train[col], X_test[col])
    drift_rows.append({
        "feature": col,
        "type": "numeric",
        "psi": psi_val,
        "ks_stat": ks_stat,
        "ks_pvalue": ks_p
    })

# Categorical drift: PSI on category proportions
def cat_psi(train_col, test_col, eps=1e-6):
    train_dist = train_col.value_counts(normalize=True)
    test_dist = test_col.value_counts(normalize=True)
    all_cats = train_dist.index.union(test_dist.index)

    train_p = train_dist.reindex(all_cats).fillna(0).values
    test_p = test_dist.reindex(all_cats).fillna(0).values

    train_p = np.clip(train_p, eps, 1)
    test_p = np.clip(test_p, eps, 1)
    return np.sum((train_p - test_p) * np.log(train_p / test_p))

for col in categorical_features:
    psi_val = cat_psi(X_train[col], X_test[col])
    drift_rows.append({
        "feature": col,
        "type": "categorical",
        "psi": psi_val,
        "ks_stat": np.nan,
        "ks_pvalue": np.nan
    })

drift_report = pd.DataFrame(drift_rows).sort_values(by="psi", ascending=False)
drift_report.head(15)


,feature,type,psi,ks_stat,ks_pvalue
11,State,categorical,0.118563,NaN,NaN
2,Total day minutes,numeric,0.029434,0.034357,0.540301
8,Total intl minutes,numeric,0.024893,0.044230,0.238984
7,Total night calls,numeric,0.021604,0.031412,0.653327
4,Total eve minutes,numeric,0.020770,0.052737,0.098528
0,Account length,numeric,0.015361,0.027489,0.802619
5,Total eve calls,numeric,0.014102,0.031684,0.643434
3,Total day calls,numeric,0.013467,0.029131,0.741476
13,International plan,categorical,0.005815,NaN,NaN
9,Total intl calls,numeric,0.005202,0.022557,0.942236


In [42]:
def psi_flag(v):
    if v < 0.10: return "low"
    if v < 0.25: return "moderate"
    return "high"

summary = drift_report.copy()
summary["psi_level"] = summary["psi"].apply(psi_flag)
summary.groupby(["type", "psi_level"]).size()


type         psi_level
categorical  low           3
             moderate      1
numeric      low          11
dtype: int64

In [43]:
import numpy as np
import pandas as pd

thresholds = np.linspace(0.05, 0.95, 19)

train_costs = []
test_costs = []

# ---- Train (CV probabilities already computed: y_proba) ----
for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)
    train_costs.append(expected_cost(y, y_pred_t))

# ---- Test ----
for t in thresholds:
    y_test_pred_t = (y_test_proba >= t).astype(int)
    test_costs.append(expected_cost(y_test, y_test_pred_t))

cost_curve = pd.DataFrame({
    "threshold": thresholds,
    "train_cost": train_costs,
    "test_cost": test_costs
})

cost_curve


,threshold,train_cost,test_cost
0,0.05,25550,6390
1,0.10,23920,5990
2,0.15,22440,5410
3,0.20,20890,4990
4,0.25,19620,4790
5,0.30,18920,4530
6,0.35,18330,4320
7,0.40,18530,4100
8,0.45,18380,4150
9,0.50,19040,4200


In [44]:
best_test_row = cost_curve.loc[cost_curve["test_cost"].idxmin()]
best_test_row


threshold         0.4
train_cost    18530.0
test_cost      4100.0
Name: 7, dtype: float64